In [1]:
#!/usr/bin/env python
from __future__ import print_function

# from mpi4py import MPI

import sqlite3

import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdMolTransforms as rdmt
from func_timeout import FunctionTimedOut,func_set_timeout

In [2]:
args = {}
args["AMOUNT_OF_ELEM"]  = 10
args["NUM_CONFS"] = 100
args["FILENAME"] = './data/tox21_10k_data_all_no_salts'
args["DATA"]='./data/tox21_10k_data_all_no_salts.csv'

In [12]:


#number of conformers created for every molecule
global NUM_CONFS
NUM_CONFS=args["NUM_CONFS"]

#amount of chemical elements taking into account
global AMOUNT_OF_ELEM
AMOUNT_OF_ELEM=args["AMOUNT_OF_ELEM"]


def enum(*sequential, **named):
    """Handy way to fake an enumerated type in Python
    http://stackoverflow.com/questions/36932/how-can-i-represent-an-enum-in-python
    """
    enums = dict(zip(sequential, range(len(sequential))), **named)
    return type('Enum', (), enums)

# Define MPI message tags
tags = enum('READY', 'DONE', 'EXIT', 'START')

# # Initializations and preliminaries
# comm = MPI.COMM_WORLD   # get MPI communicator object
# size = comm.size        # total number of processes
# rank = comm.rank        # rank of this process
# status = MPI.Status()   # get MPI status object

#creating array of elements in each chemical compound to use
def create_element_dict(data,amount=9,treshold=10, add_H=False):
    elements={}
    norm=0
    for j,smile in enumerate(data['SMILES']):
        molecule=Chem.MolFromSmiles(smile)
        try:
            molecule=Chem.AddHs(molecule)
        except:
            data = data[data['SMILES'] != smile]
            continue

        for i in range(molecule.GetNumAtoms()):
            atom = molecule.GetAtomWithIdx(i)
            element=atom.GetSymbol()
            norm+=1
            if element in elements.keys():
                elements[element]+=1
            else:
                elements[element]=1
    for key in elements.keys():
        elements[key]/=norm
    from collections import OrderedDict
    dd = OrderedDict(sorted(elements.items(), key=lambda x: x[1]))
    elements=list(dd.keys())[-amount:]  
    elements=dict((elem,i) for i, elem in enumerate(elements))  
    if not add_H:
        del elements['H']
    return elements

#read dataset
data=pd.read_csv(args["DATA"])
global elements
elements=create_element_dict(data,amount=AMOUNT_OF_ELEM)